## Moving Averages for a Singular Index - Testing Notebook

We will create a program that performs moving averages on the SPY stock. The main purpose of this program is to build familiarity for how the different libraries interact and how backtesting can be done on a optimized solution.

This will be done in three steps:
1. import data using openbb
2. build sk-learn model to optimize mean reversion strategy
3. walk-forward backtesting on historical data using vectorbt

Afterward, we will perform a fourth step in order to assess the effectiveness of our strategy:
4. validation of results (through a simulation on data outside the overall training period)

In other programs, we will add a fourth step (train sk-learn model with inputs from ta-lib indicators), but the ta-lib indicators are not needed for the moving averages strategy.

In [2]:
from openbb_terminal.sdk import openbb
import talib
import vectorbt as vbt
import numpy as np
import pandas as pd
import sklearn

INFO:openbb_terminal.cryptocurrency.onchain.bitquery_model:START
INFO:openbb_terminal.cryptocurrency.onchain.bitquery_model:END


Forecasting Toolkit is disabled. To use the Forecasting features please install the toolkit following the instructions here: https://my.openbb.co/app/sdk/installation

Portfolio Optimization Toolkit is disabled. To use the Optimization features please install the toolkit following the instructions here: https://my.openbb.co/app/sdk/installation

INFO:openbb_terminal.loggers:Logging configuration finished
INFO:openbb_terminal.loggers:Logging set to ['file', 'posthog']
INFO:openbb_terminal.loggers:Verbosity set to 20
INFO:openbb_terminal.loggers:LOGFORMAT: %(levelname)s-%(appName)s-%(commitHash)s-%(appId)s-%(sessionId)s-%(userId)s-%(asctime)s-%(name)s-%(funcName)s-%(lineno)s-%(message)s
INFO:openbb_terminal.core.log.generation.settings_logger:STARTUP: {"PREFERENCES": {"PLOT_BACKEND": null, "PLOT_DPI": 100, "PLOT_HEIGHT": 500, "PLOT_WIDTH": 800, "PLOT_HEIGHT_PERCENTAGE": 50.0, "PLOT_WIDTH_PERCENTAGE": 70.0, "PLOT_OPEN_EXPORT": false, "PLOT_ENABLE_PYWRY": true, "PLOT_PYWRY_WIDTH": 1400, "PLOT_PYWRY_HEIGHT": 762, "FILE_OVERWRITE": false, "SHOW_VERSION": true, "RETRY_WITH_LOAD": false, "USE_TABULATE_DF": true, "USE_INTERACTIVE_DF": true, "USE_CLEAR_AFTER_CMD": false, "USE_DATETIME": true, "USE_PROMPT_TOOLKIT": true, "USE_PLOT_AUTOSCALING": false, "ENABLE_THOUGHTS_DAY": false, "ENABLE_QUICK_EXIT": false, "OPEN_REPORT_AS_HTML": true, 

### Import Data Using OpenBB

In [3]:
# grab data

ohlcv = openbb.stocks.load(symbol="SPY", start_date="2016-03-01", end_date="2023-01-30")
print(ohlcv)
print(type(ohlcv))

INFO:openbb_terminal.stocks.stocks_helper:START
INFO:openbb_terminal.stocks.stocks_helper:{"INPUT": {"start_date": "2016-03-01", "interval": "1440", "end_date": "2023-01-30", "prepost": "False", "source": "YahooFinance", "weekly": "False", "monthly": "False", "verbose": "True", "symbol": "SPY", "chart": "False"}, "VIRTUAL_PATH": "stocks.load", "CHART": false}


Loading Daily data for SPY with starting period 2016-03-01.

INFO:openbb_terminal.stocks.stocks_helper:END


                  Open        High         Low       Close   Adj Close  \
date                                                                     
2016-03-01  169.364375  172.143556  168.878022  172.056702  172.056702   
2016-03-02  171.735327  172.881728  171.309762  172.829620  172.829620   
2016-03-03  172.647224  173.524408  172.056657  173.507034  173.507034   
2016-03-04  173.706784  174.870571  172.855667  174.071548  174.071548   
2016-03-07  173.124897  174.627395  173.046736  174.210510  174.210510   
...                ...         ...         ...         ...         ...   
2023-01-24  392.908312  395.144316  391.686886  394.208557  394.208557   
2023-01-25  390.022202  394.701089  387.667969  394.356323  394.356323   
2023-01-26  397.094688  398.857898  394.041092  398.690430  398.690430   
2023-01-27  397.616759  402.049389  397.400051  399.606506  399.606506   
2023-01-30  396.769618  399.064752  394.287356  394.592712  394.592712   

               Volume  Dividends  Sto

In [21]:
# instantiate 200 day mean with close prices

closes = ohlcv['Close']

data = pd.DataFrame()

data['Close'] = closes

data['Moving Average (10)'] = closes.rolling(window=10).mean()
data['Moving Average (15)'] = closes.rolling(window=15).mean()
data['Moving Average (20)'] = closes.rolling(window=20).mean()
data['Moving Average (25)'] = closes.rolling(window=25).mean()
data['Moving Average (30)'] = closes.rolling(window=30).mean()
data['Moving Average (35)'] = closes.rolling(window=35).mean()
data['Moving Average (40)'] = closes.rolling(window=40).mean()
data['Moving Average (45)'] = closes.rolling(window=45).mean()
data['Moving Average (50)'] = closes.rolling(window=50).mean()

data['Close After 1 Week'] = closes.shift(periods=-7)
data['1 Week After Change'] = data['Close After 1 Week'] - data['Close']

data = data['2016-12-16':'2023-01-01']

print(data)

                 Close  Moving Average (10)  Moving Average (15)  \
date                                                               
2016-12-16  199.642075           198.615742           197.146238   
2016-12-19  200.076797           199.132495           197.521314   
2016-12-20  200.848602           199.664693           197.922561   
2016-12-21  200.289703           199.885237           198.317712   
2016-12-22  199.943741           200.022678           198.737422   
...                ...                  ...                  ...   
2022-12-23  377.177429           381.833871           383.730448   
2022-12-27  375.690033           380.284775           382.655884   
2022-12-28  371.020996           377.972662           381.646574   
2022-12-29  377.699493           376.580392           381.126290   
2022-12-30  376.704620           376.046606           380.338348   

            Moving Average (20)  Moving Average (25)  Moving Average (30)  \
date                                  

### Build Various SK-Learn Mean Reversion Models

In [22]:
# building and testing models

def build_model(prices, parameters):
    X = prices[parameters].values
    y = prices['1 Week After Change'].values

    lasso_model = sklearn.linear_model.Lasso(alpha=1.0)  # You can specify the regularization strength (alpha)
    lasso_model.fit(X, y)

    return lasso_model

def test_model(model, dates, parameters):
    return model.predict(dates[parameters].values)

In [23]:
# establishing different model parameters:
# model 1: close, rolling average
# model 2: close, rolling average, 3 wk change
# model 3: close, rolling average, rolling min, rolling max
# model 4: close, rolling average, rolling min, rolling max, 3 wk change
# model 5: close, rolling average, lower boundary, upper boundary
# model 6: close, rolling average, lower boundary, upper boundary, 3 wk change
# model 7: close, rolling average, lower boundary indicator, upper boundary indicator
# model 8: close, rolling average, lower boundary indicator, upper boundary indicator, 3 wk change

def get_parameters(version):
    m1params = ['Moving Average (10)','Moving Average (20)']
    m2params = ['Moving Average (10)','Moving Average (30)']
    m3params = ['Moving Average (10)','Moving Average (40)']
    m4params = ['Moving Average (10)','Moving Average (50)']
    m5params = ['Moving Average (20)','Moving Average (30)']
    m6params = ['Moving Average (20)','Moving Average (40)']
    m7params = ['Moving Average (20)','Moving Average (50)']
    m8params = ['Moving Average (30)','Moving Average (40)']
    m9params = ['Moving Average (30)','Moving Average (50)']
    m10params = ['Moving Average (40)','Moving Average (50)']
    params = [m1params,m2params,m3params,m4params,m5params,m6params,m7params,m8params,m9params,m10params]
    return params[version]

### Generating Entry and Exit Signals

In [24]:
# entry and exit signals are a pandas series of boolean values indicating when a stock should be bought and sold

def get_signals(pred_returns):
    signals = pd.DataFrame()
    
    signals['Base Entries'] = np.where(pred_returns > 0, True, False)
    signals['Base Exits'] = np.where(pred_returns < 0, True, False)

    max=pred_returns.max()
    min=pred_returns.min()
    pos_avg=np.mean(pred_returns[np.where(pred_returns>0)])
    neg_avg=np.mean(pred_returns[np.where(pred_returns<0)])

    signals['0.25 Entries'] = np.where(pred_returns > 0.25*max, True, False)
    signals['0.25 Exits'] = np.where(pred_returns < 0.25*min, True, False)

    signals['0.5 Entries'] = np.where(pred_returns > 0.5*max, True, False)
    signals['0.5 Exits'] = np.where(pred_returns < 0.5*min, True, False)

    signals['0.25-0.75 Entries'] = np.where((pred_returns > 0.25*max) & (pred_returns < 0.75*max), True, False)
    signals['0.25-0.75 Exits'] = np.where((pred_returns < 0.25*min) & (pred_returns > 0.75*min), True, False)

    signals['0.5-0.8 Entries'] = np.where((pred_returns > 0.5*max) & (pred_returns < 0.8*max), True, False)
    signals['0.5-0.8 Exits'] = np.where((pred_returns < 0.5*min) & (pred_returns > 0.8*min), True, False)

    signals['Above Average Entries'] = np.where(pred_returns > pos_avg, True, False)
    signals['Above Average Exits'] = np.where(pred_returns < neg_avg, True, False)

    signals['Around Average Entries'] = np.where((pred_returns > 0.5*pos_avg) & (pred_returns < 2*pos_avg), True, False)
    signals['Around Average Exits'] = np.where((pred_returns < 0.5*neg_avg) & (pred_returns > 2*neg_avg), True, False)

    return signals

### Backtest SK-Learn Model with VectorBT

In [25]:
figure = data['Close'].vbt.rolling_split(n=12, window_len=475, set_lens=(95,),left_to_right=False,plot=True)
figure.update_layout(width=640,height=360)
figure.show()

print(list(range(0,7)))

[0, 1, 2, 3, 4, 5, 6]


In [47]:
# backtesting code

(in_sample_prices,in_sample_dates), (out_sample_prices,out_sample_dates) = data['Close'].vbt.rolling_split(n=12, window_len=475, set_lens=(95,),left_to_right=False)

signal_names = [('Base Entries','Base Exits'),('0.25 Entries','0.25 Exits'),('0.5 Entries','0.5 Exits'),('0.25-0.75 Entries','0.25-0.75 Exits'),('0.5-0.8 Entries','0.5-0.8 Exits'),('Above Average Entries','Above Average Exits'),('Around Average Entries','Around Average Exits')]

models_idx = list(range(0,10))
signals_idx = list(range(0,7))
returns_idx = pd.MultiIndex.from_tuples([(m,s) for m in models_idx for s in signals_idx], names=['model_version','signal_level'])
returns = pd.DataFrame(columns=returns_idx)
sharpe = pd.DataFrame(columns=returns_idx)

for m in range(0,10):
    for s in range(0,7):
        split_idx_list = list(range(0,12))
        split_idx = pd.MultiIndex.from_tuples([(x,) for x in split_idx_list], names=['split_idx'])
        entries = pd.DataFrame(columns=split_idx)
        exits = pd.DataFrame(columns=split_idx)
        for i in range(0,12):
            model = build_model(data[in_sample_dates[i][0]:in_sample_dates[i][-1]], get_parameters(m))
            pred = test_model(model, data[out_sample_dates[i][0]:out_sample_dates[i][-1]], get_parameters(m))
            signals = get_signals(pred)
            entries.loc[:, i] = signals[signal_names[s][0]]
            exits.loc[:, i] = signals[signal_names[s][1]]
        pf=vbt.Portfolio.from_signals(out_sample_prices, entries, exits, freq='1d', direction='both')
        res=pd.DataFrame({'Total Return': pf.total_return(), 'Sharpe Ratio': pf.sharpe_ratio()})
        returns.loc[:, (m,s)] = pf.total_return()
        sharpe.loc[:, (m,s)] = pf.sharpe_ratio()
        # print(f"Model {m+1} ({signal_names[s][0]}, {signal_names[s][1]}):")
        # print(res)

# print(returns)

returns_sums = (returns.sum()).sort_values(ascending=False)
sharpe_averages = sharpe.mean()

results = pd.DataFrame(index=returns_sums.index)
results['returns sums'] = returns_sums.values
results['sharpe avgs'] = sharpe_averages.reindex(results.index).values
print(results.head(60))
print(results.shape)

                            returns sums  sharpe avgs
model_version signal_level                           
6             5                 0.761075     0.811522
              2                 0.757668     0.870404
5             5                 0.747977     0.791090
6             4                 0.718420     0.820566
9             5                 0.709203     0.575761
8             5                 0.626322     0.484476
0             5                 0.579711     0.618233
4             5                 0.575160     0.494722
7             5                 0.568730     0.322995
5             2                 0.510368     0.483049
4             2                 0.482482     0.456424
0             2                 0.473821     0.454555
5             4                 0.471120     0.433210
8             2                 0.466352     0.433771
3             2                 0.452915     0.322635
4             4                 0.443234     0.406585
2             5             

### Validation of Results

In [48]:
final_closes = (openbb.stocks.load(symbol="SPY", start_date="2020-01-01", end_date="2023-01-01"))['Close']

final_data = pd.DataFrame()

final_data['Close'] = final_closes

final_data['Moving Average (10)'] = final_closes.rolling(window=10).mean()
final_data['Moving Average (15)'] = final_closes.rolling(window=15).mean()
final_data['Moving Average (20)'] = final_closes.rolling(window=20).mean()
final_data['Moving Average (25)'] = final_closes.rolling(window=25).mean()
final_data['Moving Average (30)'] = final_closes.rolling(window=30).mean()
final_data['Moving Average (35)'] = final_closes.rolling(window=35).mean()
final_data['Moving Average (40)'] = final_closes.rolling(window=40).mean()
final_data['Moving Average (45)'] = final_closes.rolling(window=45).mean()
final_data['Moving Average (50)'] = final_closes.rolling(window=50).mean()

final_data['Close After 1 Week'] = final_closes.shift(periods=-7)
final_data['1 Week After Change'] = final_data['Close After 1 Week'] - final_data['Close']

final_data = final_data['2021-06-21':'2022-12-20']

print(final_data.shape)

INFO:openbb_terminal.stocks.stocks_helper:START
INFO:openbb_terminal.stocks.stocks_helper:{"INPUT": {"start_date": "2020-01-01", "interval": "1440", "end_date": "2023-01-01", "prepost": "False", "source": "YahooFinance", "weekly": "False", "monthly": "False", "verbose": "True", "symbol": "SPY", "chart": "False"}, "VIRTUAL_PATH": "stocks.load", "CHART": false}


Loading Daily data for SPY with starting period 2020-01-02.

INFO:openbb_terminal.stocks.stocks_helper:END


(380, 12)


In [49]:
future_closes = (openbb.stocks.load(symbol="SPY", start_date="2021-01-01", end_date="2023-05-18"))['Close']

future_data = pd.DataFrame()

future_data['Close'] = future_closes

future_data['Moving Average (10)'] = future_closes.rolling(window=10).mean()
future_data['Moving Average (15)'] = future_closes.rolling(window=15).mean()
future_data['Moving Average (20)'] = future_closes.rolling(window=20).mean()
future_data['Moving Average (25)'] = future_closes.rolling(window=25).mean()
future_data['Moving Average (30)'] = future_closes.rolling(window=30).mean()
future_data['Moving Average (35)'] = future_closes.rolling(window=35).mean()
future_data['Moving Average (40)'] = future_closes.rolling(window=40).mean()
future_data['Moving Average (45)'] = future_closes.rolling(window=45).mean()
future_data['Moving Average (50)'] = future_closes.rolling(window=50).mean()

future_data = future_data["2023-01-01":"2023-05-18"]

print(future_data.shape)

INFO:openbb_terminal.stocks.stocks_helper:START
INFO:openbb_terminal.stocks.stocks_helper:{"INPUT": {"start_date": "2021-01-01", "interval": "1440", "end_date": "2023-05-18", "prepost": "False", "source": "YahooFinance", "weekly": "False", "monthly": "False", "verbose": "True", "symbol": "SPY", "chart": "False"}, "VIRTUAL_PATH": "stocks.load", "CHART": false}


Loading Daily data for SPY with starting period 2021-01-04.

INFO:openbb_terminal.stocks.stocks_helper:END


(95, 10)


In [50]:
models_idx = list(range(0,10))
signals_idx = list(range(0,7))
future_returns_idx = pd.MultiIndex.from_tuples([(m,s) for m in models_idx for s in signals_idx], names=['model_version','signal_level'])
future_returns = pd.DataFrame(columns=future_returns_idx)

for m in range(0,10):
    for s in range(0,7):
        model = build_model(final_data, get_parameters(m))
        pred = test_model(model, future_data, get_parameters(m))
        signals = get_signals(pred)
        entries = signals[signal_names[s][0]]
        exits = signals[signal_names[s][1]]
        pf=vbt.Portfolio.from_signals(future_closes["2023-01-01":"2023-05-18"].values, entries, exits, freq='1d', direction='both')
        future_returns.loc[:, (m,s)] = [pf.total_return()]

print(future_returns)
future_returns_sums = future_returns.sum()
print(future_returns_sums)

results['future returns'] = future_returns_sums.reindex(results.index).values
print(results.head(60))
print(results['future returns'].mean())

model_version         0                                                   \
signal_level          0         1         2         3        4         5   
0              0.068002  0.070268  0.087666  0.029553  0.06578  0.087666   

model_version                   1                      ...         8  \
signal_level          6         0         1         2  ...         4   
0              0.022785  0.136956  0.158806  0.229683  ...  0.122859   

model_version                             9                               \
signal_level          5         6         0         1        2         3   
0              0.092864  0.106499  0.012574 -0.018373 -0.05929 -0.018373   

model_version                               
signal_level         4         5         6  
0             -0.05929 -0.044643 -0.013147  

[1 rows x 70 columns]
model_version  signal_level
0              0               0.068002
               1               0.070268
               2               0.087666
               3   